In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from torch.amp import autocast
from pytorch_memlab import MemReporter
import gc

In [2]:
model_directory = r"Z:/llmfile/Llama-2-7B-hf/models--meta-llama--Llama-2-7b-hf/snapshots/01c7f73d771dfac7d292323805ebc428287df4f9"

model = AutoModelForCausalLM.from_pretrained(model_directory)
tokenizer = AutoTokenizer.from_pretrained(model_directory)

# 初始输入
input_text = "Hello"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

reporter = MemReporter(model)

# 前向传播
print("==ForwardTrainingMemoryUsage==")
model.train()
with autocast("cuda"): # 启用混合精度
    outputs = model(input_ids)
    loss = outputs.loss
    reporter.report()  # 前向传播后的内存使用情况

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

==ForwardTrainingMemoryUsage==
Element type                                            Size  Used MEM
-------------------------------------------------------------------------------
Storage on cpu
Tensor0                                        (1, 2, 11008)    86.00K
Tensor1                                        (1, 2, 11008)    86.00K
Tensor2                                        (1, 2, 11008)    86.00K
Tensor3                                        (1, 2, 11008)    86.00K
Tensor4                                         (1, 2, 4096)    32.00K
Tensor5                                            (1, 2, 1)   512.00B
Tensor6                                         (1, 2, 4096)    32.00K
Tensor7                                         (1, 2, 4096)    32.00K
Tensor8                                         (1, 2, 4096)    32.00K
Tensor9                                      (1, 32, 2, 128)     0.00B
Tensor10                                      (1, 1, 2, 128)     1.00K
Tensor11              

C:\Users\xxc13\anaconda3\Lib\site-packages\pytorch_memlab\mem_reporter.py:65: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  tensors = [obj for obj in objects if isinstance(obj, torch.Tensor)]
C:\Users\xxc13\anaconda3\Lib\site-packages\pytorch_memlab\mem_reporter.py:95: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  fact_numel = tensor.storage().size()


In [3]:
# 清除前向传播的变量，确保不占用内存
del outputs, loss
gc.collect()  # 强制清理内存

46